In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00


In [98]:
from datasets import load_dataset
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Dataset

In [72]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [118]:
dataset = load_dataset("lex_glue", "ecthr_b")
#print(dataset['train'][0]['labels'])
x_train, x_test = pd.DataFrame(dataset['train']), pd.DataFrame(dataset['test'])
print(x_train, x_test)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


                                                   text     labels
0     [11.  At the beginning of the events relevant ...        [4]
1     [9.  The applicant is the monarch of Liechtens...  [8, 3, 9]
2     [9.  In June 1949 plots of agricultural land o...        [3]
3     [8.  In 1991 Mr Dušan Slobodník, a research wo...  [6, 8, 5]
4     [9.  The applicant is an Italian citizen, born...     [8, 3]
...                                                 ...        ...
8995  [5.  The applicant was born in 1960 and lives ...     [3, 4]
8996  [5.  The applicant was born in 1946 and is cur...         []
8997  [5.  The applicants are Russian nationals who,...  [0, 1, 2]
8998  [8.  In all cases the applicants brought civil...     [3, 9]
8999  [4.  The applicant was born in 1971 and lives ...        [3]

[9000 rows x 2 columns]                                                   text        labels
0    [5.  The applicant is a journalist for DN.no, ...           [6]
1    [5.  The applicant was born 

In [126]:
print(len(x_train['text'][0][0]))

688


In [127]:
#Preprocessing
#Lowercasing
x_train['text'] = x_train['text'].apply(lambda x: [' '.join(sent_tokenize(re.sub(r'[^a-zA-Z0-9\s]', '', sentence).lower())) for sentence in x])
x_test['text'] = x_test['text'].apply(lambda x: [' '.join(sent_tokenize(re.sub(r'[^a-zA-Z0-9\s]', '', sentence).lower())) for sentence in x])

In [114]:
#Removing stop words
stop_words = set(stopwords.words('english'))
x_train['text'] = x_train['text'].apply(lambda x: [' '.join([word for word in word_tokenize(sentence) if word not in stop_words]) for sentence in x])
x_test['text'] = x_test['text'].apply(lambda x: [' '.join([word for word in word_tokenize(sentence) if word not in stop_words]) for sentence in x])

In [135]:
print(len(x_train['text'][0]))
print(x_train['text'][0][0])

83
11  at the beginning of the events relevant to the application k had a daughter p and a son m born in 1986 and 1988 respectively ps father is x and ms father is v from march to may 1989 k was voluntarily hospitalised for about three months having been diagnosed as suffering from schizophrenia from august to november 1989 and from december 1989 to march 1990 she was again hospitalised for periods of about three months on account of this illness in 1991 she was hospitalised for less than a week diagnosed as suffering from an atypical and undefinable psychosis it appears that social welfare and health authorities have been in contact with the family since 1989


In [138]:
train_data, train_labels = x_train['text'], x_train['labels']
test_data, test_labels = x_test['text'], x_test['labels']

In [139]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# BERT Model
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()  # Set the model to evaluation mode

# Custom Dataset class for BERT embeddings
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [145]:
train_dataset = TextDataset(train_data, train_labels)
test_dataset = TextDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)
print(train_dataset[0])

{'input_ids': tensor([ 101, 2340, 2012,  ...,    0,    0,    0]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'label': tensor([4])}


In [149]:
def extract_bert_embeddings(loader):
    embeddings = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Extracting BERT embeddings"):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            outputs = model(input_ids, attention_mask=attention_mask)
            embeddings.append(outputs['last_hidden_state'].mean(dim=1))  # Average pooling
    return torch.cat(embeddings)

In [154]:
train_embeddings = extract_bert_embeddings(train_loader)
test_embeddings = extract_bert_embeddings(test_loader)

# Train a simple classifier on top of BERT embeddings
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_embeddings, train_labels)

# Make predictions on the validation set
val_predictions = classifier.predict(test_embeddings)

# Evaluate accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Validation Accuracy: {accuracy}")

Extracting BERT embeddings:   0%|          | 0/4500 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [372] at entry 0 and [902] at entry 1